Necessary installation for running in Google Colab

In [0]:
!apt-get update
!apt-get -qq install python-opengl -y
!apt-get -qq -y install xvfb ffmpeg
!pip -q install pyvirtualdisplay
!pip -q install piglet
!pip -q install gym[box2d]


# Augmented Random Search



In [0]:
import gym
import numpy as np
from gym import wrappers

Utility class Hp with Hyperparameters

In [0]:
class Hp():
    # Hyperparameters
    def __init__(self,
                 episode_length=2000,
                 learning_rate=0.05,
                 num_deltas=16,
                 num_best_deltas=16,
                 noise=0.03,
                 seed=1,
                 env_name='BipedalWalker-v2',
                 record_every=50):
        self.episode_length = episode_length
        self.learning_rate = learning_rate
        self.num_deltas = num_deltas
        self.num_best_deltas = num_best_deltas
        assert self.num_best_deltas <= self.num_deltas
        self.noise = noise
        self.seed = seed
        self.env_name = env_name
        self.record_every = record_every


Utility class Normalizer that normalize data

In [0]:
class Normalizer():
    # Normalizes the inputs
    def __init__(self, nb_inputs):
        self.n = np.zeros(nb_inputs)
        self.mean = np.zeros(nb_inputs)
        self.mean_diff = np.zeros(nb_inputs)
        self.var = np.zeros(nb_inputs)
        
    def observe(self, x):
        self.n += 1.0
        last_mean = self.mean.copy()
        self.mean += (x - self.mean) / self.n
        self.mean_diff += (x - last_mean) * (x - self.mean)
        self.var = (self.mean_diff / self.n).clip(min = 1e-2)

    def normalize(self, inputs):
        self.observe(inputs)
        obs_mean = self.mean
        obs_std = np.sqrt(self.var)
        return (inputs - obs_mean) / obs_std
  

Agent trained in the environment

In [0]:
class ArsTrainer():
    def __init__(self, env, input_size=None, output_size=None, hp=None, normalizer=None, monitor_dir=None):
        self.env = env
        self.input_size = input_size or self.env.observation_space.shape[0]
        self.output_size = output_size or self.env.action_space.shape[0]
        self.weights = np.zeros((self.output_size, self.input_size))
        self.hp = hp or Hp()
        self.normalizer = normalizer or Normalizer(self.input_size)
        self.cur_step = 0;
        self.set_monitor(monitor_dir)
        self.record_video = False

    def set_monitor(self, monitor_dir=None):
        #use this method if you want to record the episode
        #set the folder where the recorded video will be stored 
        if monitor_dir is not None:
            should_record = lambda i: self.record_video
            self.env = wrappers.Monitor(self.env, monitor_dir, video_callable=should_record, force=True)

    def learning_rate(self, decay=0.005):
        return self.hp.learning_rate / (1 + decay * self.cur_step)

    def train(self, n_steps):
        for step in range(n_steps):
            self.cur_step += 1
            # initialize the random noise deltas and the positive/negative rewards
            deltas = self.generate_deltas()
            positive_rewards = np.zeros(self.hp.num_deltas)
            negative_rewards = np.zeros(self.hp.num_deltas)
            # play an episode each with positive deltas and negative deltas, collect rewards
            for i in range(self.hp.num_deltas):
                positive_rewards[i] = self.play_episode(self.weights + self.hp.noise * deltas[i])
                negative_rewards[i] = self.play_episode(self.weights - self.hp.noise * deltas[i])

            # Compute the standard deviation of all rewards
            sigma_rewards = np.array(positive_rewards + negative_rewards).std()

            # Sort the rollouts by the max(r_pos, r_neg) and select the deltas with best rewards
            scores = {k: max(r_pos, r_neg) for k, (r_pos, r_neg) in enumerate(zip(positive_rewards, negative_rewards))}
            order = sorted(scores.keys(), key=lambda x: scores[x], reverse=True)[:self.hp.num_best_deltas]
            rollouts = [(positive_rewards[k], negative_rewards[k], deltas[k]) for k in order]
            # Update the policy
            self.update_weights(rollouts, sigma_rewards)

            # Only record video during evaluation, every n steps
            if step % self.hp.record_every == 0:
                self.record_video = True
            # Play an episode with the new weights and print the score
            reward_evaluation = self.play_episode(self.weights, train=False)
            print('Step: ', step, 'Reward: ', reward_evaluation)
            self.record_video = False

    def update_weights(self, rollouts, sigma_rewards):
        # sigma_rewards is the standard deviation of the rewards
        step = np.zeros(self.weights.shape)
        for r_pos, r_neg, delta in rollouts:
            step += (r_pos - r_neg) * delta
        self.weights += self.learning_rate() / (self.hp.num_best_deltas * sigma_rewards) * step

    def play_episode(self, theta=None, train=True):
        # play one episode of game
        if theta is None:
            theta = self.weights
        obs = self.env.reset()
        sum_reward = 0
        episode = 0
        while True:
            episode += 1
            # choose action using theta
            action = self.predict(obs, theta)
            obs, reward, done, _ = self.env.step(action)
            sum_reward += reward
            if done:
                break
            # break if reached max number of episodes
            if not self.record_video and episode >= self.hp.episode_length:
                break
        return sum_reward

    def predict(self, inp, theta):
        # predict action from input using theta
        inp = self.normalizer.normalize(inp)
        return theta @ inp

    def generate_deltas(self):
        return np.random.randn(self.hp.num_deltas, *self.weights.shape)


Train model

In [0]:
hp = Hp()
env = gym.make('BipedalWalker-v2')
trainer = ArsTrainer(env)
trainer.train(1000)

##Train and record video and record 

Connect to Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive/')
DIR_PATH = 'drive/My Drive/Colab Notebooks/BipedalWalker/'

create directories

In [0]:
import os
def mkdir(base, name):
    path = os.path.join(base, name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

In [0]:
videos_dir = mkdir(DIR_PATH, 'videos')
monitor_dir = mkdir(videos_dir, 'bi')

create display

In [0]:
# Start virtual display
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
# os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)


train and save video to Google Drive

In [0]:
hp = Hp()
env = gym.make('BipedalWalker-v2')
trainer = ArsTrainer(env,monitor_dir=monitor_dir)

In [0]:
trainer.train(20)